# Conversation Chain with Memory

LLMs are stateless, which means they process each incoming query in isolation, without considering previous interactions.
To overcome this limitation, LangChain offers a standard interface for memory, a variety of memory implementations and examples
of chains and agents that employ memory:
* ConversationBufferMemory: Stores the entire conversation history as a single string
* ConversationBufferWindowMemory: Maintains a memory window that keeps a limited number of past interactions.
* ConversationSummaryMemory: Holds a summary of previous converations.
* ConversationSummaryBufferMemory: This method combines the features: of ConversationSummaryMemory and ConversationBufferWindowMemory.Stores the conversation's summary and buffer.


In [1]:
import sys
sys.path.append("../src/mrag")

In [29]:
from langchain.memory import ConversationBufferMemory,ConversationBufferWindowMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate,PromptTemplate
from langchain import ConversationChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate

from llm_instances import get_embeddings_instance, get_llm_instance

* LLM instance

In [3]:
llm_model = get_llm_instance()

* Embeddings

In [4]:

embedding_function = get_embeddings_instance()

##  ConversationChain 

By default, LangChain's ConversationChain has a simple type of memory that remembers all previous inputs/outputs and adds them to the context that is passed.
It can also remember what the initial message were after 3 questions.

In [5]:

conversation_= ConversationChain(
    llm=llm_model,
    verbose=True
)


In [6]:
conversation_.predict(input=" what's The capital of France?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human:  what's The capital of France?
AI:

> Finished chain.


'The capital of France is Paris. It is the largest city in the country and is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. Paris is also a major center for art, fashion, and cuisine.'

In [7]:

conversation_.predict(input=" and Italy?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human:  what's The capital of France?
AI: The capital of France is Paris. It is the largest city in the country and is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. Paris is also a major center for art, fashion, and cuisine.
Human:  and Italy?
AI:

> Finished chain.


"The capital of Italy is Rome. It is a city with a rich history, known for its ancient ruins such as the Colosseum and the Roman Forum, as well as its art and architecture, including the Vatican City and St. Peter's Basilica. Rome is also famous for its delicious cuisine, including pasta, pizza, and gelato."

In [15]:

conversation_.predict(input=" and Spain?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human:  what's The capital of France?
AI: The capital of France is Paris. It is the largest city in the country and is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. Paris is also a major center for art, fashion, and cuisine.
Human:  and Italy?
AI: The capital of Italy is Rome. It is a city with a rich history, known for its ancient ruins such as the Colosseum and the Roman Forum, as well as its art and architecture, including the Vatican City and St. Peter's Basilica. Rome is also famous for its delicious cuisine, including pasta, pizza, and gelato.
Human:  and Spain?
AI:

> Finished chain.


'The capital of Spain is Madrid. It is the largest city in the country and is known for its rich cultural heritage, including the Royal Palace, the Prado Museum, and the Retiro Park. Madrid is also famous for its vibrant nightlife and delicious tapas.'

##  ConversationBufferMemory

In [22]:


prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm_model, verbose=True)

In [23]:
user_message = "Tell me how to write an endpoint in python."
response = conversation(user_message)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a human and an AI.
Human: Tell me how to write an endpoint in python.

> Finished chain.
{'input': 'Tell me how to write an endpoint in python.', 'history': [HumanMessage(content='Tell me how to write an endpoint in python.'), AIMessage(content='To write an endpoint in Python, you can use a web framework such as Flask or Django. Here\'s a simple example using Flask:\n\n```python\nfrom flask import Flask\n\napp = Flask(__name__)\n\n@app.route(\'/hello\', methods=[\'GET\'])\ndef hello():\n    return \'Hello, World!\'\n\nif __name__ == \'__main__\':\n    app.run()\n```\n\nIn this example, we define a route `/hello` that responds to GET requests with the message "Hello, World!". When you run the Flask app, it will start a web server and listen for incoming requests to this endpoint.\n\nYou can then test this endpoint by making a GET request to `http://localhost:5000

In [27]:
response.keys()

dict_keys(['input', 'history', 'response'])

In [25]:
response['input']

'Tell me how to write an endpoint in python.'

In [26]:
response['history']

[HumanMessage(content='Tell me how to write an endpoint in python.'),
 AIMessage(content='To write an endpoint in Python, you can use a web framework such as Flask or Django. Here\'s a simple example using Flask:\n\n```python\nfrom flask import Flask\n\napp = Flask(__name__)\n\n@app.route(\'/hello\', methods=[\'GET\'])\ndef hello():\n    return \'Hello, World!\'\n\nif __name__ == \'__main__\':\n    app.run()\n```\n\nIn this example, we define a route `/hello` that responds to GET requests with the message "Hello, World!". When you run the Flask app, it will start a web server and listen for incoming requests to this endpoint.\n\nYou can then test this endpoint by making a GET request to `http://localhost:5000/hello` (assuming you\'re running the app locally on port 5000).')]

In [28]:
response['response']

'To write an endpoint in Python, you can use a web framework such as Flask or Django. Here\'s a simple example using Flask:\n\n```python\nfrom flask import Flask\n\napp = Flask(__name__)\n\n@app.route(\'/hello\', methods=[\'GET\'])\ndef hello():\n    return \'Hello, World!\'\n\nif __name__ == \'__main__\':\n    app.run()\n```\n\nIn this example, we define a route `/hello` that responds to GET requests with the message "Hello, World!". When you run the Flask app, it will start a web server and listen for incoming requests to this endpoint.\n\nYou can then test this endpoint by making a GET request to `http://localhost:5000/hello` (assuming you\'re running the app locally on port 5000).'

In [ ]:
#Nota: Analizamos la respuesta obtenida con  ConversationChain.Vemos que se guarda el input, el historial y la respuesta del LLM

## ConversationBufferWindowMemory

* Prompt

In [33]:
template = """Act as friendly, professional and polite expert in API specifications. 
You will provided with a question and a context. Your task is to answer the question based on the context and in the language provided.
If the question can't be answered based on the context,  say you don't know.

{chat_history}
Context: {context}
Language: {language}
Question: {question}
Answer:"""

question = "¿Hay algun servicio para crear una mascota?"
chat_history=""
language = "English"

# create a prompt template
prompt = PromptTemplate(
    input_variables=["chat_history", "context","language","question"], 
    template=template
)

In [31]:
conversation_with_memory = ConversationChain(
    llm=llm_model,
    memory = ConversationBufferWindowMemory(k=3, return_messages=True)
)

In [32]:
conversation_with_memory(question)

{'input': '¿Hay algun servicio para crear una mascota?',
 'history': [],
 'response': 'Sí, hay varios servicios en línea que te permiten crear tu propia mascota virtual. Puedes personalizar su apariencia, personalidad y habilidades. Algunos de estos servicios incluso te permiten interactuar con tu mascota a través de juegos y actividades virtuales.'}